# Домашнее задание
Дообучение энкодерных моделей

**Цель:**

В этом задании вы поработаете с энкодерными трансформерными моделями (например, BERT) и дообучите их для решения различных задач обработки естественного языка (NLP).

**Описание / пошаговая инструкция выполнения домашнего задания:**


1. **Обработка данных:**

- В дополнительных материалах к уроку найдите датасет с отзывами о ресторанах (restaurants_reviews.jsonl).
- Разбейте данные на train/val/test, отложив по 15% в test и val. Не забудьте зафиксировать random_state. В качестве целевой переменной возьмите общий отзыв из колонки general.
- Оставьте только отзывы с рейтингом general равным 1, 3 и 5. Для удобства перекодируйте лейблы 1, 3 и 5 в метки 0, 1, 2.


2. **Дообучение энкодерных моделей:**

- Возьмите 3 модели:

    - https://huggingface.co/sberbank-ai/ruBert-base/ или https://huggingface.co/sberbank-ai/ruBert-large/
    - https://huggingface.co/cointegrated/rubert-tiny2
    - https://huggingface.co/google-bert/bert-base-multilingual-cased

- Дообучите каждую модель на train части данных. Обучение прекращайте, когда модель выходит на плато по метрике на валидации.
- Возьмите итоговый чекпоинт (версию, с минимальным loss на валидации) и замерьте качество на test. В качестве метрики используйте accuracy.


3. **Анализ результатов:**

- Составьте таблицу с результатами для каждой модели, включающую:

    - количество эпох до достижения минимального значения loss на валидационной выборке;
    - время, затрачиваемое на одну итерацию обучения;
    - общее время дообучения;

- Проведите анализ полученных результатов и опишите выводы в Markdown в ноутбуке.


In [ ]:
import gc, time
import pandas as pd
pd.options.display.max_colwidth = 300
import numpy as np
import torch
from tqdm.auto import tqdm, trange
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from torch.optim import Adam
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

#1. Обработка данных

##1.1.Загрузка данных: датасет с отзывами о ресторанах (restaurants_reviews.jsonl)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# df = pd.read_json('./restaurants_reviews.jsonl', lines=True)
df = pd.read_json('/content/drive/MyDrive/restaurants_reviews.jsonl', lines=True)
df[20:25]

##1.2.Оставляем только отзывы с рейтингом "general" равным 1, 3 и 5, используем булевую маску (True/False), True для строк, где general равен 1, 3 или 5

In [ ]:
df = df[df['general'].isin([1,3,5])] # перезапишем df, оставляя только строки с general равным 1, 3 или 5.
df['general'].value_counts() # посмотрим распределение

## 1.3.Перекодируем лейблы 1, 3 и 5 в метки 0, 1, 2.

In [ ]:
rating_map = {1:0, 3:1, 5:2}
df['general'] = df['general'].map(rating_map)

In [ ]:
df['general'].value_counts()

##1.4.Разбиваем данные на train/val/test, отложив по 15% в test и val. В качестве целевой переменной берем общий отзыв из колонки general.

In [ ]:
# Создаём объект Dataset из библиотеки Hugging Face datasets на основе DataFrame df
dataset_f = Dataset.from_dict({'text': df.text, 'label': df.general}) # целевая переменная - "general" (Y)
dataset_f

In [ ]:
# Первичное разделение на обучающую + валидационную (85%) и тестовую (15%) части
train_test_split = dataset_f.train_test_split(test_size=0.15, seed=42)  # Указываем seed для воспроизводимости
train_val_split = train_test_split["train"]  # Временный набор для дальнейшего деления
test_dataset = train_test_split["test"]

In [ ]:
# Вторичное разделение на финальные обучающую (70%) и валидационную (15%) части
# Размер валидации: 0.15 / 0.85 ≈ 0.1765 от оставшихся данных
final_split = train_val_split.train_test_split(test_size=0.1765, seed=42)
train_dataset = final_split["train"]
val_dataset = final_split["test"]

In [ ]:
print("Размеры после разделения средствами datasets:")
print(f"  Обучающая выборка (train): {len(train_dataset)} примеров")
print(f"  Валидационная выборка (validation): {len(val_dataset)} примеров")
print(f"  Тестовая выборка (test): {len(test_dataset)} примеров")

In [ ]:
# Создаем удобный словарь датасетов
data = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset,
})

In [ ]:
# Чистим память
del dataset_f, train_test_split, train_val_split, final_split

In [ ]:
data['train'][2]

# 2. Дообучение энкодерных моделей

    - https://huggingface.co/sberbank-ai/ruBert-base/
    - https://huggingface.co/cointegrated/rubert-tiny2
    - https://huggingface.co/google-bert/bert-base-multilingual-cased

## 2.1 ruBERT-base

In [ ]:
#Загрузка модели
b_base_model = 'ai-forever/ruBert-base'

In [ ]:
#Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(b_base_model)
tokenizer

In [ ]:
#Подготовка текстовых данных для моделей трансформеров.Токенизация текстовых данных с помощью библиотеки Transformers
data_tokenized = data.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), batched=True, remove_columns=['text'])
data_tokenized

In [ ]:
#Убедимся, что токенизация работает правильно: выведем третий элемент из тренировочной части токенизированного датасета
print(data_tokenized['train'][2])

In [ ]:
# Cоздаём объект DataCollatorWithPadding для автоматического дополнения (padding) батчей данных до одинаковой длины.
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#Создаём три DataLoader'а (для загруки данных батчами, их автоматического перемешивания и коллатора)для обучения, валидации и тестирования модели
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=8, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['validation'], shuffle=False, batch_size=8, collate_fn=collator)
test_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=8, collate_fn=collator)

In [ ]:
#Создаём модель для классификации последовательностей текста на основе предобученной модели
model = AutoModelForSequenceClassification.from_pretrained(b_base_model, num_labels=3)
model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
#Перемещаем модель на выбранное вычислительное устройство (GPU или CPU)
model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)  #малая скорость обучения и след. меньше риск "перепрыгнуть" оптимум
optimizer

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#Полный цикл обучения нейронной сети с сохранением лучшей модели
best_eval_loss = float('inf') #начальное значение для поиска минимума

losses = []
epoch_train_loss = []
epoch_eval_loss = []
epoch_train_time = []
train_time = []
start = time.time()
for epoch in trange(20):
    pbar = tqdm(train_dataloader)
    model.train()
    for i, batch in enumerate(pbar):
        out = model(**batch.to(model.device))
        out.loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(out.loss.item())
        train_time.append(time.time() - start)
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')
    epoch_train_loss.append(np.mean(losses[-100:]))

    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    val_time = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
        val_time.append(time.time() - start)
    epoch_eval_loss.append(np.mean(eval_losses))
    epoch_train_time.append(elapsed := time.time() - start)
    val_loss = np.mean(eval_losses)
    print('Epoch:', epoch+1, 'Train Loss', np.mean(losses[-100:]), 'Eval Loss', val_loss, 'Accuracy', np.mean(np.array(eval_targets) == eval_preds), 'Time:', elapsed)

    #сохраняем лучшую модель
    if val_loss < best_eval_loss:
        best_eval_loss = val_loss
        torch.save(model.state_dict(), model.name_or_path.split('/')[1]+'.saved.weights.pt')

In [ ]:
#Находим индекс (номер) лучшей эпохи обучения и соответствующую минимальную ошибку валидации
epoch_eval_loss.index(min(epoch_eval_loss)), min(epoch_eval_loss)

In [ ]:
#Находим общее время обучения до лучшей эпохи (эпохи с минимальной ошибкой валидации)
epoch_train_time[epoch_eval_loss.index(min(epoch_eval_loss))]

In [ ]:
epoch_train_time[-1]

In [ ]:
#вычисление среднего времени обучения на одну эпоху
epoch_train_time[-1]/len(epoch_train_time)

In [ ]:
# Построение графика потерь на обучающей и валидационной выборках
plt.figure(figsize=(10, 4))
plt.plot(np.arange(1, 21, 1), epoch_train_loss, color='darkblue', linewidth=2, label='Train Loss')
plt.plot(np.arange(1, 21, 1), epoch_eval_loss, color='red', linewidth=2, label='Eval Loss', linestyle='--')

plt.xticks(np.arange(1, 21, 1))
plt.xlabel('Эпоха', fontsize=12)
plt.ylabel('Потери', fontsize=12)
plt.title('Динамика потерь по эпохам', fontsize=14, pad=15)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()  # Автоматическая подгонка отступов
plt.show()

##Загрузим лучшую сохраненную модель

In [ ]:
#Загрузка лучшей сохраненной модели
path = model.name_or_path.split('/')[1]+'.saved.weights.pt'
model.load_state_dict(torch.load(path))

In [ ]:
model.to(device)

In [ ]:
#Выполняем оценку модели на тестовом наборе данных.
model.eval()
test_losses = []
test_preds = []
test_targets = []

for batch in tqdm(test_dataloader):
    with torch.no_grad():
            out = model(**batch.to(model.device))
    test_losses.append(out.loss.item())
    test_preds.extend(out.logits.argmax(1).tolist())
    test_targets.extend(batch['labels'].tolist())

print('Eval Loss', np.mean(test_losses), 'Accuracy', np.mean(np.array(test_targets) == test_preds))

In [ ]:
accuracy_score(test_targets, test_preds)

In [ ]:
confusion_matrix(test_targets, test_preds)

## Оценка модели

In [ ]:
results = {}

In [ ]:
def quality(etime, vloss):
    """
        etime: списокаккумулированного времени train для каждой эпохи;
        vloss: список validation losses для каждой эпохи;
    """
    # количество эпох до достижения минимального значения loss на валидационной выборке,
    min_epoch_num = vloss.index(min(vloss))
    #общее время дообучения
    total_train_time = etime[min_epoch_num]
    #время, затрачиваемое на одну итерацию обучения
    avg_train_step_time = etime[-1]/len(etime)
    print(f"min_epoch_num: {min_epoch_num+1}, avg_train_step_time: {avg_train_step_time}, total_train_time: {total_train_time} ")
    return [min_epoch_num+1, round(avg_train_step_time, 2), round(total_train_time, 2)]

In [ ]:
def get_model_results(etime, vloss, dataloader, model):
    """
        dataloader: Dataloader для  модели;
        model: Model для test;
    """
    # количество эпох до достижения минимального значения loss на валидационной выборке,
    min_epoch_num = vloss.index(min(vloss))
    #общее время дообучения
    total_train_time = etime[min_epoch_num]
    #время, затрачиваемое на одну итерацию обучения
    avg_train_step_time = etime[-1]/len(etime)

    model.eval()
    test_losses = []
    test_preds = []
    test_targets = []

    for batch in tqdm(test_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        test_losses.append(out.loss.item())
        test_preds.extend(out.logits.argmax(1).tolist())
        test_targets.extend(batch['labels'].tolist())

    accuracy_metric = np.mean(np.array(test_targets) == test_preds)

    print(f"min_epoch: {min_epoch_num+1}, epoch_time: {avg_train_step_time}, total_train_time: {total_train_time}, accuracy: {accuracy_metric}")
    return [min_epoch_num+1, round(avg_train_step_time, 2), round(total_train_time, 2), round(accuracy_metric,4)]

In [ ]:
quality(epoch_train_time, epoch_eval_loss)

In [ ]:
get_model_results(epoch_train_time, epoch_eval_loss, test_dataloader, model)

Сохраним результаты

In [ ]:
results['ruBERT-base'] = get_model_results(epoch_train_time, epoch_eval_loss, test_dataloader, model)

In [ ]:
pd.DataFrame(results, index=['Epoch num','Epoch avg time','Total train time','Accuracy']).T

## 2.2 ruBERT-tiny2

In [ ]:
b_base_model = 'cointegrated/rubert-tiny2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(b_base_model)
tokenizer

In [ ]:
data_tokenized = data.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), batched=True, remove_columns=['text'])
data_tokenized

In [ ]:
print(data_tokenized['train'][2])

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=16, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['validation'], shuffle=False, batch_size=16, collate_fn=collator)
test_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=16, collate_fn=collator)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(b_base_model, num_labels=3)
model

In [ ]:
model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-5)
optimizer

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
best_eval_loss = float('inf')

losses = []
epoch_train_loss = []
epoch_eval_loss = []
epoch_train_time = []
train_time = []
start = time.time()
for epoch in trange(20):
    pbar = tqdm(train_dataloader)
    model.train()
    for i, batch in enumerate(pbar):
        out = model(**batch.to(model.device))
        out.loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(out.loss.item())
        train_time.append(time.time() - start)
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')
    epoch_train_loss.append(np.mean(losses[-100:]))

    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    val_time = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
        val_time.append(time.time() - start)
    epoch_eval_loss.append(np.mean(eval_losses))
    epoch_train_time.append(elapsed := time.time() - start)
    val_loss = np.mean(eval_losses)
    print('Epoch:', epoch+1, 'Train Loss', np.mean(losses[-100:]), 'Eval Loss', val_loss, 'Accuracy', np.mean(np.array(eval_targets) == eval_preds), 'Time:', elapsed)

    #сохранение лучшей модели
    if val_loss < best_eval_loss:
        best_eval_loss = val_loss
        torch.save(model.state_dict(), model.name_or_path.split('/')[1]+'.saved.weights.pt')

In [ ]:
# Построение графика потерь на обучающей и валидационной выборках
plt.figure(figsize=(10, 4))
plt.plot(np.arange(1, 21, 1), epoch_train_loss, color='darkblue', linewidth=2, label='Train Loss')
plt.plot(np.arange(1, 21, 1), epoch_eval_loss, color='red', linewidth=2, label='Eval Loss', linestyle='--')
plt.xticks(np.arange(1, 21, 1))
plt.xlabel('Эпоха', fontsize=12)
plt.ylabel('Потери', fontsize=12)
plt.title('Динамика потерь по эпохам', fontsize=14, pad=15)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()  # Автоматическая подгонка отступов
plt.show()

##Загрузим лучшую сохраненную модель

In [ ]:
#loading the best saved model
path = model.name_or_path.split('/')[1]+'.saved.weights.pt'
path

In [ ]:
model.load_state_dict(torch.load(path))

In [ ]:
model.to(device)

### Протестируем и сохраним результаты

In [ ]:
get_model_results(epoch_train_time, epoch_eval_loss, test_dataloader, model)

In [ ]:
results['ruBERT-tiny2'] = get_model_results(epoch_train_time, epoch_eval_loss, test_dataloader, model)

## 3. Анализ результатов

Расчет модели bert-base-multilingual-cased не был выполнен из-за того, что закончились вычислительные ресурсы. Но это не мешает сделать итоговые выводы.

In [ ]:
pd.DataFrame(results, index = ['Epoch num','Epoch avg time','Total train time','Accuracy']).T.sort_values(by=['Accuracy'], ascending=False)

**Вывод:**

между моделями наблюдается компромисс между точностью и скоростью. Выбор лучшей модели будет зависеть от конкретных целей проекта.

**Точность:**

- ruBert-base точнее (0.90). Разница в 2% в классификации бывает важной и может оправдать использование более тяжелой модели (670 МБ).
- rubert-tiny2 показывает отличный результат для своего класса (точность =0.88, размер модели около 125МБ). Уступая всего 2%, она показывает высокую эффективность архитектуры, адаптированной для русского языка.

**Скорость и эффективность обучения**

Здесь преимущество rubert-tiny2 абсолютно очевидно:

- Время эпохи меньше в ~11.5 раз (12 сек. против 127 сек.). Это позволяет гораздо быстрее проводить эксперименты, отлаживать код и перебирать гиперпараметры.
- Общее время обучения меньше в ~7,7 раз (70 сек. против 536 сек.). Tiny2 достигла плато за большее число эпох, но благодаря скорости каждой эпохи обучилась значительно быстрее.
- ruBert-base достигла плато быстрее (за 4 эпохи), но цена каждой эпохи очень высока.

Для предварительных расчетов в Google Colab я выбрала бы модель tiny2

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Установим nbconver
!pip install nbconvert -q

file_path = '/content/drive/MyDrive/HW_2_Advanc_free.ipynb'


# "очистим" блокнот, удалив выводы и проблемные метаданные
!jupyter nbconvert "{file_path}" --to notebook --output "{file_path}" --clear-output --inplace

[NbConvertApp] Converting notebook /content/drive/MyDrive/HW_2_Advanc_free.ipynb to notebook
[NbConvertApp] Writing 35550 bytes to /content/drive/MyDrive/HW_2_Advanc_free.ipynb
